**извлечение признаков из текста на естественном языке**

кластеризатор текстов

Евгений Борисов borisov.e@solarl.ru

## библиотеки

In [1]:
import re
# import gzip
import numpy as np
import numpy.random as rng
import pandas as pd
from tqdm import tqdm

np.set_printoptions(precision=2) # вывод на печать чисел до 2 знака
pd.options.display.max_colwidth = 200 

tqdm.pandas()

/usr/lib/python3.8/site-packages/tqdm/std.py:703: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
from sklearn import __version__ as SKLEARN_VERSION
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

from sklearn.feature_extraction.text import TfidfVectorizer

from pymorphy2 import MorphAnalyzer

SKLEARN_VERSION

'0.23.2'

## тексты

In [3]:
# загружаем тексты
data = pd.read_pickle('../data/news.pkl.gz')
print('записей:',len(data))

записей: 3196


In [4]:
data.sample(2)

,text,tag
2777,Специалисты Apple планируют начать широкомасштабную кампанию по борьбе со спамом в стандартном для «яблочных» гаджетов приложении «Календарь» из macOS и iOS. В последнее время пользователи активно...,tech
3088,"Департамент межнациональных отношений министерства регионального развиия РФ\nКОМИИНФОРМ. Новости Госслужащие должны быть осторожнее в формулировках,\nособенно если дело касается межэтнических отно...",politics


## токенизация и очистка

In [5]:
# лемматизация и очистка с помощью пакета морфологического анализа

morph = MorphAnalyzer()

# NOUN	существительное
# VERB	глагол
# ADJF	прилагательное
def word_normalizer(w, pos_types=('NOUN','VERB','ADJF')):
    if not morph.word_is_known(w): return ''
    p = morph.parse(w)[0] 
    return p.normal_form if (p.tag.POS in pos_types) else ''

def string_normalizer(s):
    return re.sub(r'[\W]+',' ',s.lower().replace('ё','е'))

def text_normalizer(s):
    return [ word_normalizer(w) for w in string_normalizer(s).split() ]

In [6]:
data['ctext'] = data['text'].progress_apply(text_normalizer).apply(' '.join)

100%|██████████| 3196/3196 [01:39<00:00, 32.07it/s]


In [7]:
data.sample(10)

,text,tag,ctext
3095,"Наш край (Красноярск) Подхвати спасательный круг В Туруханске прошло выездное\nзаседание Гражданской ассамблеи Красноярского края, на котором обсуждали\nвопросы возрождения традиционного образа жи...",social,наш край красноярск подхватить спасательный круг туруханск выездной заседание гражданский ассамблея красноярский край который обсуждать вопрос возрождение традиционный образ жизнь коренной мало...
3083,"Интерфакс Религия (interfax-religion.ru) Патриарх Кирилл посетит Камчатку\nПатриарх Московский и всея Руси Кирилл 20-23 сентября, как ожидается, побывает\nна Камчатке. Москва. 16 июля. ИНТЕРФАКС ...",social,интерфакс религия патриарх кирилл посетить камчатка патриарх московский весь русь кирилл сентябрь ожидаться побывать камчатка москва июль интерфакс патриарх московский весь русь кирилл ...
719,"Столичный житель обвиняется в распространении наркотиков по всей Минской области\n\n8 декабря 2016 в 23:39\n\nВячеслав Будкевич, БелаПАН\n\nПрокуратура Минской области направила в суд уголовное де...",incident,столичный житель обвиняться распространение наркотик весь минский область декабрь вячеслав будкевич прокуратура минский область направить суд уголовный дело отношение летний житель с...
2183,В Кемеровской области на работавших на месте ДТП сотрудников экстренных служб наехал автомобиль. Об этом сообщается на сайте регионального главка МВД.\n\nАвария произошла на 106-м километре дороги...,incident,кемеровский область место дтп сотрудник экстренный служба наехать автомобиль сообщаться сайт региональный главка мвд авария произойти метр километр дорога ленинск кузнецкий новокузнецк ...
2198,"На востоке Москвы загорелся склад пиломатериалов, сообщили в столичном ГУ МЧС России. Сообщение о возгорании одноэтажного складского здания поступило в ведомство 13 декабря в 8:43.\n\nПрибывшие на...",incident,восток москва загореться склад пиломатериал сообщить столичный мчс россия сообщение возгорание складской здание поступить ведомство декабрь место чп спасательный бригада обнаружить в...
1240,"""Мы испытываем его, а он испытывает нас"". Как работает водитель-испытатель БЕЛАЗа\n\n1 декабря 2016 в 13:30\n\nПавел Мурашко, Фото: Вадим Замировский / AUTO.TUT.BY\n\nВодители-испытатели БЕЛАЗов к...",auto,испытывать испытывать работать водитель испытатель декабрь павел фото вадим водитель испытатель казаться какой нереальный человек такой махина тот карьер чаща левостор...
1868,"Дональд Трамп, избранный президент США, заявил, что оставит свой бизнес в срок до 20 января 2017 года, то есть до даты инаугурации, после чего сосредоточится на президентской работе.\n\nТакое реше...",politics,дональд трамп избранный президент сша заявить оставить свой бизнес срок январь год дата инаугурация сосредоточиться президентский работа такой решение будущий глава государство принять ...
1117,"Хацкевич назвал слухами информацию о его возможном назначении в киевское ""Динамо""\n\n6 декабря 2016 в 21:11\n\nSPORT.TUT.BY\n\nУволенный с поста главного тренера сборной Беларуси Александр Хацкеви...",sport,назвать слух информация возможный назначение киевский динамо декабрь пост главный тренер сборный беларусь александр сообщить футбол информация возможный назначение киевский дин...
2777,Специалисты Apple планируют начать широкомасштабную кампанию по борьбе со спамом в стандартном для «яблочных» гаджетов приложении «Календарь» из macOS и iOS. В последнее время пользователи активно...,tech,специалист планировать широкомасштабный кампания борьба спам стандартный яблочный гаджет приложение календарь последний время пользователь жаловаться рассылка сообщение почтовый ящик...
2480,​12 декабря Борис Гребенщиков и группа «Аквариум» с симфоническим оркестром сорвали аншлаг на концерте в Новосибирске.\n\n\n\n\n\nКонцерт «БГ SYMPHONIA» Гребенщиков исполнил на сцене ДКЖ.\n\n\n\n\...,culture,декабрь борис гребенщик группа аквариум симфонический оркестр сорвать аншлаг концерт новосибирск концерт гребенщик исполнить сцена приехать сотоварищ казахстан преодолеть 

##  Vectorizer

In [8]:
tf = TfidfVectorizer(use_idf=True,norm='l2').fit( data['ctext'] )

In [9]:
len(tf.vocabulary_)

22128

## формируем датасет

In [10]:
X = tf.transform( data['ctext'] )
X.shape

(3196, 22128)

## кластеризируем

In [11]:
from sklearn.cluster import DBSCAN

clust = DBSCAN(eps=.95,min_samples=10).fit(X)

In [12]:
data['cluster'] = clust.labels_

In [13]:
data['cluster'].drop_duplicates().count()

36

## тестируем

In [14]:
from matplotlib import pyplot as plt

In [15]:
cluster_descr = pd.concat([
        data[['cluster','tag']].groupby(['cluster'])['tag'].count(),
        data[['cluster','tag']].groupby(['cluster'])['tag'].apply(lambda s: set(s)).apply(' '.join)
    ],axis=1).reset_index()

cluster_descr.columns = ['cluster','count','tags']

cluster_descr

,cluster,count,tags
0,-1,2658,sport culture incident social realty auto tech economics woman science reclama politics health
1,0,17,politics
2,1,21,politics
3,2,15,politics
4,3,34,politics
5,4,28,politics
6,5,35,politics
7,6,13,incident
8,7,25,politics
9,8,12,sport


In [16]:
n = 0
print(cluster_descr.query(f'cluster=={n}'))
data.query(f'cluster=={n}')[['text','tag']].sample(5)

   cluster  count      tags
1        0     17  politics


,text,tag
1676,"МОСКВА, 13 декабря. /ТАСС/. Экс-президент СССР Михаил Горбачев допускает возможность появления в границах бывшего Советского Союза нового Союзного государства.\n\n""Советского Союза - нет, а Союза ...",politics
1406,"Михаил Горбачев в интервью, приуроченном к 25-й годовщине распада СССР, ответил на вопрос, возможно ли его восстановление: «Советского Союза — нет, а Союза — да». «В прежних границах и с тем же со...",politics
1673,"Горбачев увидел возможность восстановления Союза в прежних границах\n\nОбъединение, по его словам, может стать полностью добровольным\n\nПервый президент СССР Михаил Горбачев заявил о возможности ...",politics
1403,"Горбачев заявил о возможном создании в границах бывшего СССР нового Союза\n\nТелеканал «Россия – Вологда»\n\nЭкс-президент Советского Союза Михаил Горбачев заявил, что в границах бывшего СССР може...",politics
479,"Более половины россиян сожалеют о распаде СССР\n\n5 декабря 2016 в 12:36\n\nDeutsche Welle\n\nБольшинство россиян сожалеет о распаде СССР, но только каждый восьмой хотел бы его восстановления в пр...",politics


In [17]:
n = 1
print(cluster_descr.query(f'cluster=={n}'))
data.query(f'cluster=={n}')[['text','tag']].sample(5)

   cluster  count      tags
2        1     21  politics


,text,tag
1376,"Заявления о возможной российской угрозе странно слышать от хорватских властей, однако Москва готова обсудить аргументы Загреба, заявил глава МИД Сергей Лавров.\r\n\r\nРанее в СМИ появились высказы...",politics
1599,"Министр иностранных дел РФ назвал заявления странными, но подчеркнул, что при наличии конкретных фактов готов обсудить их с хорватским лидером\n\nГлава МИД РФ Сергей Лавров ответил на слова презид...",politics
1600,"Лавров прокомментировал «российскую угрозу» на Балканах\n\nГлава МИД России Сергей Лавров заявил, что ему странно слышать об угрозе, которую Москва несет для Загреба. По словам министра, он хорошо...",politics
1593,В МИД прокомментировали заявление президента Хорватии об угрозе войны на Балканах. Ранее СМИ опубликовали заявление президента Хорватии Колинды Грабар-Китарович об угрозе военного конфликта на Бал...,politics
1845,"Если у хорватского президента есть конкретные факты, подтверждающие эти заявления, то Россия готова эти факты обсудить и как-то прокомментировать, указал глава МИД РФ.\n\nСтранно слышать заявления...",politics


In [18]:
n = 2
print(cluster_descr.query(f'cluster=={n}'))
data.query(f'cluster=={n}')[['text','tag']].sample(5)

   cluster  count      tags
3        2     15  politics


,text,tag
1864,"Избранный президент США Дональд Трамп в ходе телефонного разговора с главным кандидатом на пост госсекретаря Миттом Ромни сообщил, что Ромни не будет занимать эту должность.\n\nПосле разговора экс...",politics
1860,Место главного дипломата Америки точно не светит экс-кандидату на пост президента США в 2012 году Митту Ромни. С мечтой возглавить дипломатический корпус известный своими антироссийскими высказыва...,politics
1386,"Дональд Трамп позвонил Ромни и уведомил, что тот не рассматривается в качестве претендента на эту должность. Об этом сообщил телеканал CNN, а затем информацию подтвердил сам Митт Ромни в Facebook....",politics
1388,"Митт Ромни расстался с мыслью стать госсекретарем США\n\nПо данным СМИ, Трамп объявит имя нового главы Госдепа во вторник утром, главным претендентом на пост является глава Exxon Mobil Рекс Тиллер...",politics
1380,"Избранный президент США Дональд Трамп позвонил Митту Ромни и сообщил, что тот не будет новым госсекретарём. Об этом сообщает CNN со ссылкой на источник в переходной команде республиканца.\n\nРеспу...",politics


In [19]:
n = 3
print(cluster_descr.query(f'cluster=={n}'))
data.query(f'cluster=={n}')[['text','tag']].sample(5)

   cluster  count      tags
4        3     34  politics


,text,tag
1854,"Президент России Владимир Путин в интервью японской телекомпании Nippon TV заявил, что у России нет территориальных проблем с Японией: «Это Япония считает, что у нее есть территориальные проблемы,...",politics
1574,У России нет территориальных проблем с Японией — они есть только по мнению Японии. Об этом заявил президент РФ Владимир Путин в интервью телекомпании Nippon TV.\n\n«У нас нет территориальных пробл...,politics
1850,"Президент России Владимир Путин заявил о том, что Москва хочет полной нормализации отношений с Токио, передает РИА «Новости» со ссылкой на японские СМИ.\n\n«Мы хотим полной нормализации отношений ...",politics
1870,"Во вторник, 13 декабря, МИД Японии официально объявил программу визита в страну президента РФ Владимира Путина.\n\nКак стало известно, глава нашего государства прибудет в аэропорт префектуры Ямагу...",politics
1414,"В МИД Японии обнародовали программу российского президента во время рабочей поездки.\n\nПутин и Абэ примут участие в ""бизнес-диалоге""\n\nЯпонское внешнеполитическое ведомство озвучило программу пр...",politics
